In [16]:

from flask import Flask, jsonify
import pandas as pd
import datetime as dt
import numpy as np
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from flask import Flask, jsonify

app = Flask(__name__)



In [4]:
precipitation_df = pd.read_csv("C:/Users/natha/OneDrive/Documents/Data Analytics Classwork/Week 10 AdvSQL/Module10SQLalchemy/Resources/hawaii_measurements.csv")
stations_df = pd.read_csv("C:/Users/natha/OneDrive/Documents/Data Analytics Classwork/Week 10 AdvSQL/Module10SQLalchemy/Resources/hawaii_stations.csv")



In [12]:
def get_temperature_observations():
    most_active_station = 'USC00519281'
    
    latest_date = precipitation_df['date'].max()
    one_year_ago = pd.to_datetime(latest_date) - pd.DateOffset(years=1)
    
    tobs_data = precipitation_df.loc[(precipitation_df['station'] == most_active_station) & 
                                      (precipitation_df['date'] >= one_year_ago) & 
                                      (precipitation_df['date'] <= latest_date), ['date', 'tobs']]
    
    tobs_dict = tobs_data.set_index('date')['tobs'].to_dict()

    return tobs_dict


def calculate_temps_start(start_date):
   
    selected_data = precipitation_df.loc[precipitation_df['date'] >= start_date, 'tobs']

    tmin = selected_data.min()
    tavg = selected_data.mean()
    tmax = selected_data.max()

    results = {
        'TMIN': tmin,
        'TAVG': tavg,
        'TMAX': tmax
    }

    return results


def calculate_temps_range(start_date, end_date):
    
    selected_data = precipitation_df.loc[(precipitation_df['date'] >= start_date) & (precipitation_df['date'] <= end_date), 'tobs']

    tmin = selected_data.min()
    tavg = selected_data.mean()
    tmax = selected_data.max()

    results = {
        'TMIN': tmin,
        'TAVG': tavg,
        'TMAX': tmax
    }

    return results


In [15]:
@app.route("/")
def welcome():
    return (
        f"Welcome to the Climate API!<br/>"
        f"Available routes:<br/>"
        f"/api/v1.0/precipitation<br/>"
        f"/api/v1.0/stations<br/>"
        f"/api/v1.0/tobs<br/>"
        f"/api/v1.0/<start><br/>"
        f"/api/v1.0/<start>/<end><br/>"
    )

if "/api/v1.0/precipitation" not in app.url_map._rules_by_endpoint:
    @app.route("/api/v1.0/precipitation")
    def precipitation():
        precipitation_data_dict = precipitation_df.groupby("date").agg({"prcp": "sum"}).to_dict()["prcp"]
        return jsonify(precipitation_data_dict)

@app.route("/api/v1.0/stations")
def stations():
    stations_list = stations_df.to_dict(orient="records")
    return jsonify(stations_list)

@app.route("/api/v1.0/tobs")
def tobs():
    temperature_observations = get_temperature_observations()
    return jsonify(temperature_observations)

@app.route("/api/v1.0/<start>")
def start_date(start):
    results = calculate_temps_start(start)
    return jsonify(results)

@app.route("/api/v1.0/<start>/<end>")
def start_end_date(start, end):
    results = calculate_temps_range(start, end)
    return jsonify(results)

if __name__ == "__main__":
    app.run(debug=True)


AssertionError: View function mapping is overwriting an existing endpoint function: welcome